# Transitive and Intransitive Verbs

## Transitive and Intransitive

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import csv

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [6]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("He left after cleaning up.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'left',
   'description': '[ARG0: He] [V: left] [ARGM-TMP: after cleaning up] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O']},
  {'verb': 'cleaning',
   'description': '[ARG0: He] left after [V: cleaning] up .',
   'tags': ['B-ARG0', 'O', 'O', 'B-V', 'O', 'O']}],
 'words': ['He', 'left', 'after', 'cleaning', 'up', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [8]:
d = ["He left after cleaning up."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'left',
     'description': '[ARG0: He] [V: left] [ARGM-TMP: after cleaning up] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O']},
    {'verb': 'cleaning',
     'description': '[ARG0: He] left after [V: cleaning] up .',
     'tags': ['B-ARG0', 'O', 'O', 'B-V', 'O', 'O']}],
   'words': ['He', 'left', 'after', 'cleaning', 'up', '.']}],
 array([1.]))

In [9]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [11]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [12]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} left after cleaning up.", meta=True,nsamples=150, remove_duplicates=True)

In [13]:
with open('/datasets/both_intransitive_transitive_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/both_intransitive_transitive_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    0 (0.0%)
['Benjamin', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Michelle', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Brian', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Joseph', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Anne', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Samuel', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Edwin', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Simon', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Brian', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Kate', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Sam', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Margaret', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Leslie', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Rebecca', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Nick', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Florence', 'left', 'after', 'cleaning', 'up', '.

In [13]:
d = ["He left the gift on the table."]
pred = predict_and_conf(d)
pred


([{'verbs': [{'verb': 'left',
     'description': '[ARG0: He] [V: left] [ARG1: the gift] [ARG2: on the table] .',
     'tags': ['B-ARG0',
      'B-V',
      'B-ARG1',
      'I-ARG1',
      'B-ARG2',
      'I-ARG2',
      'I-ARG2',
      'O']}],
   'words': ['He', 'left', 'the', 'gift', 'on', 'the', 'table', '.']}],
 array([1.]))

In [12]:
objects = ['gift', 'book', 'keys', 'wallet', 'phone', 'card']

In [14]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['objects']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1= Expect.single(found_arg1_people)

In [15]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} left the {object} on the table.", object = objects, meta=True,nsamples=150, remove_duplicates=True)
# initialize a test object
with open('/datasets/both_intransitive_transitive2_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/both_intransitive_transitive2_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    0 (0.0%)
['Patricia', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [ True]
['Carolyn', 'left', 'the', 'card', 'on', 'the', 'table', '.'] [ True]
['Victoria', 'left', 'the', 'book', 'on', 'the', 'table', '.'] [ True]
['Dorothy', 'left', 'the', 'book', 'on', 'the', 'table', '.'] [ True]
['Mark', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [ True]
['Kathleen', 'left', 'the', 'book', 'on', 'the', 'table', '.'] [ True]
['Adam', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [ True]
['Jack', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [ True]
['Sarah', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [ True]
['Rebecca', 'left', 'the', 'wallet', 'on', 'the', 'table', '.'] [ True]
['Matt', 'left', 'the', 'card', 'on', 'the', 'table', '.'] [ True]
['Grace', 'left', 'the', 'gift', 'on', 'the', 'table', '.'] [ True]
['Andrew', 'left', 'the', 'card', 'on', 'the', 'table', '.'] [ True]
['Susan', 'left', 'the', '

# Transitive verb

In [16]:
d = ["She loves cats."]
pred = predict_and_conf(d)
pred


([{'verbs': [{'verb': 'loves',
     'description': '[ARG0: She] [V: loves] [ARG1: cats] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['She', 'loves', 'cats', '.']}],
 array([1.]))

In [17]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [18]:

def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [19]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [20]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} loves cats.", meta=True,nsamples=150, remove_duplicates=True)
# initialize a test object
with open('/datasets/transitive_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/transitive_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    0 (0.0%)
['Pamela', 'loves', 'cats', '.'] [ True]
['Lauren', 'loves', 'cats', '.'] [ True]
['Henry', 'loves', 'cats', '.'] [ True]
['Al', 'loves', 'cats', '.'] [ True]
['Jane', 'loves', 'cats', '.'] [ True]
['Rebecca', 'loves', 'cats', '.'] [ True]
['Ryan', 'loves', 'cats', '.'] [ True]
['Frank', 'loves', 'cats', '.'] [ True]
['Greg', 'loves', 'cats', '.'] [ True]
['Alfred', 'loves', 'cats', '.'] [ True]
['Richard', 'loves', 'cats', '.'] [ True]
['Rose', 'loves', 'cats', '.'] [ True]
['Fred', 'loves', 'cats', '.'] [ True]
['Philip', 'loves', 'cats', '.'] [ True]
['Katherine', 'loves', 'cats', '.'] [ True]
['Charles', 'loves', 'cats', '.'] [ True]
['James', 'loves', 'cats', '.'] [ True]
['Anna', 'loves', 'cats', '.'] [ True]
['Grace', 'loves', 'cats', '.'] [ True]
['Lauren', 'loves', 'cats', '.'] [ True]
['Bill', 'loves', 'cats', '.'] [ True]
['Ellen', 'loves', 'cats', '.'] [ True]
['Kathryn', 'loves', 'cats', '.'] [ True]
['

## Intransive verbs

In [21]:
d = ["She sang."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'sang',
     'description': '[ARG0: She] [V: sang] .',
     'tags': ['B-ARG0', 'B-V', 'O']}],
   'words': ['She', 'sang', '.']}],
 array([1.]))

In [22]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [23]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [24]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [25]:
adjectives = ['loud', 'nice', 'emotional', 'horrible', 'bad', 'decent', 'good']

In [28]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} sang.", meta=True,nsamples=20, remove_duplicates = True)
# initialize a test object
with open('/datasets/intransitive_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/intransitive_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 20 examples
Test cases:      20
Fails (rate):    0 (0.0%)
['Eric', 'sang', '.'] [ True]
['Sam', 'sang', '.'] [ True]
['Lucy', 'sang', '.'] [ True]
['Martha', 'sang', '.'] [ True]
['Sam', 'sang', '.'] [ True]
['Nicole', 'sang', '.'] [ True]
['Michelle', 'sang', '.'] [ True]
['Amy', 'sang', '.'] [ True]
['Kathleen', 'sang', '.'] [ True]
['Kelly', 'sang', '.'] [ True]
['Ray', 'sang', '.'] [ True]
['Jennifer', 'sang', '.'] [ True]
['Jennifer', 'sang', '.'] [ True]
['Tony', 'sang', '.'] [ True]
['Florence', 'sang', '.'] [ True]
['Claire', 'sang', '.'] [ True]
['Ben', 'sang', '.'] [ True]
['Nancy', 'sang', '.'] [ True]
['Anthony', 'sang', '.'] [ True]
['Jay', 'sang', '.'] [ True]


In [29]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} sang very {adj}.", adj = adjectives, meta=True,nsamples=200, remove_duplicates = True)
with open('/datasets/intransitive_adj_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/intransitive_adj_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 200 examples
Test cases:      200
Fails (rate):    0 (0.0%)
['Alison', 'sang', 'very', 'decent', '.'] [ True]
['Charles', 'sang', 'very', 'bad', '.'] [ True]
['Louise', 'sang', 'very', 'decent', '.'] [ True]
['Francis', 'sang', 'very', 'bad', '.'] [ True]
['Betty', 'sang', 'very', 'decent', '.'] [ True]
['Robert', 'sang', 'very', 'bad', '.'] [ True]
['Ray', 'sang', 'very', 'nice', '.'] [ True]
['Jennifer', 'sang', 'very', 'emotional', '.'] [ True]
['Susan', 'sang', 'very', 'nice', '.'] [ True]
['Nick', 'sang', 'very', 'emotional', '.'] [ True]
['Diana', 'sang', 'very', 'emotional', '.'] [ True]
['Fiona', 'sang', 'very', 'horrible', '.'] [ True]
['Albert', 'sang', 'very', 'emotional', '.'] [ True]
['Virginia', 'sang', 'very', 'nice', '.'] [ True]
['Eleanor', 'sang', 'very', 'horrible', '.'] [ True]
['Sarah', 'sang', 'very', 'good', '.'] [ True]
['Rebecca', 'sang', 'very', 'horrible', '.'] [ True]
['Michael', 'sang', 'very', 'good', '.'] [ True]
['Ryan', 'sang', 'very', 'good'

# End of Notebook